## Design and implement a Complete University Management System in Python using SQLite and functions (def) with the following requirements:
1. Database (SQLite)

Create the following tables:

Departments (dept_id, dept_name, hod_name)

Students (student_id, name, class, dept_id)

Subjects (subject_id, subject_name, dept_id)

Marks (mark_id, student_id, subject_id, marks)

In [227]:
import sqlite3
conn = sqlite3.connect('students.db')
cursor = conn.cursor()

In [229]:
cursor.execute('''
    CREATE TABLE departments(
        dept_id INTEGER PRIMARY KEY,
        dept_name TEXT,
        hod_name TEXT
    )
''')
conn.commit()

In [230]:
cursor.execute('''
    CREATE TABLE student(
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        stu_class TEXT,
        dept_id INTEGER,
        FOREIGN KEY(dept_id) REFERENCES departments(dept_id)
    )
''')
conn.commit()

In [231]:
cursor.execute('''CREATE TABLE subjects (
            subject_id INTEGER PRIMARY KEY ,
            subject_name TEXT,
            dept_id INTEGER,
            FOREIGN KEY(dept_id) references departments(dept_id)
)''')
conn.commit()

In [232]:
cursor.execute('''
CREATE TABLE Marks (
    mark_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    subject_id INTEGER,
    marks INTEGER,
    FOREIGN KEY(student_id) REFERENCES Students(student_id),
    FOREIGN KEY(subject_id) REFERENCES Subjects(subject_id)
)
''')
conn.commit()

## data in list for all tables

In [233]:
departments_list = [
    (1, "Data Science", "DR XX"),
    (2, "Physics", "Dr YY")
]


students_list = [
    ("AA", "DL", 1),
    ("BB", "ML", 1),
    ("CC", "Set", 2),
    ("DD", "Algebra", 2),
    ("EE", "Calcus", 1)
]
subjects_list = [
    ("Data Structures", 1),
    ("DBMS", 1),
    ("Algebra", 2),
    ("Calcus", 2)
]

marks_list = [
    (1, 1, 78), (1, 2, 81),
    (2, 1, 75), (2, 2, 80),
    (3, 1, 50), (3, 2, 45),
    (4, 1, 85), (4, 2, 90),
    (5, 1, 95), (5, 2, 92)
]

## function to insert values

In [234]:
def insert_departments(data_list):
    cursor.executemany("INSERT INTO departments (dept_id, dept_name, hod_name) VALUES (?, ?, ?)", data_list)
    conn.commit()

def insert_student(data_list):
    cursor.executemany("INSERT INTO student (name,stu_class, dept_id) VALUES (?, ?, ?)", data_list)
    conn.commit()

def insert_subjects(data_list):
    cursor.executemany("INSERT INTO subjects (subject_name, dept_id) VALUES (?, ?)", data_list)
    conn.commit()

def insert_marks(data_list):
    cursor.executemany("INSERT INTO marks (student_id, subject_id, marks) VALUES (?, ?, ?)", data_list)
    conn.commit()


cursor.execute("DELETE FROM departments")
cursor.execute("DELETE FROM student")
cursor.execute("DELETE FROM subjects")
cursor.execute("DELETE FROM marks")
conn.commit()

# inserting values to table using function

In [235]:
insert_departments(departments_list)
insert_student(students_list)
insert_subjects(subjects_list)
insert_marks(marks_list)

In [236]:
def show_studenst():
    print("\n--- Students ---")
    cursor.execute('''
        SELECT s.student_id, s.name, s.stu_class, d.dept_name
        FROM student s
        JOIN departments d ON s.dept_id = d.dept_id
    ''')
    for row in cursor.fetchall():
        print(row)
print("done")

done


cursor.execute("DROP TABLE IF EXISTS Marks")
cursor.execute("DROP TABLE IF EXISTS Subjects")
cursor.execute("DROP TABLE IF EXISTS Student")
cursor.execute("DROP TABLE IF EXISTS Departments")

##show_studenst()

In [237]:
def show_subjects():
    print("\n--- Subjects ---")
    cursor.execute('''
        SELECT subj.subject_id, subj.subject_name, d.dept_name
        FROM subjects subj
        JOIN departments d ON subj.dept_id = d.dept_id
    ''')
    for row in cursor.fetchall():
        print(row)
##show_subjects()

In [238]:
def show_marks():
    print("\n--- Marks ---")
    cursor.execute('''
        SELECT s.name, subj.subject_name, m.marks
        FROM marks m
        JOIN student s ON m.student_id = s.student_id
        JOIN subjects subj ON m.subject_id = subj.subject_id
    ''')
    for row in cursor.fetchall():
        print(row)
##show_marks()

In [239]:
def unique_subjects():
    cursor.execute('SELECT subject_name FROM subjects')
    rows = cursor.fetchall()
    return {row[0] for row in rows}
##unique_subjects()

In [240]:
def toppers():
    print("\n--- Topper in Each Subject ---")
    cursor.execute('''
        SELECT subj.subject_name, stu.name, m.marks
        FROM Marks m
        JOIN student stu ON m.student_id = stu.student_id
        JOIN subjects subj ON m.subject_id = subj.subject_id
        WHERE (subj.subject_id, m.marks) IN (
            SELECT subject_id, MAX(marks)
            FROM Marks
            GROUP BY subject_id
        )
    ''')
    subject_toppers = cursor.fetchall()
    for subject_name, student_name, max_mark in subject_toppers:
        print(f"Subject: {subject_name:<10} \n Topper: {student_name:<10} | Marks: {max_mark}")

    print("\n--- Overall Topper ---")
    cursor.execute('''
        SELECT stu.name, SUM(m.marks) as total_marks
        FROM Marks m
        JOIN Student stu ON m.student_id = stu.student_id
        GROUP BY m.student_id
        ORDER BY total_marks DESC
        LIMIT 1
    ''')
    overall_topper = cursor.fetchone()
    if overall_topper:
        name, total = overall_topper
        print(f"Overall Topper: {name} | Total Marks: {total}")


In [245]:
## print student name and marks based on student id 


def studentReports(student_id):
    

    # Get student info
    cursor.execute('''
        SELECT name FROM student WHERE student_id = ?
    ''', (student_id,))
    student = cursor.fetchone()

    if not student:
        print(f"No student found with ID {student_id}")
        return

    student_name = student[0]

    # Get subject-wise marks
    cursor.execute('''
        SELECT sub.subject_name, m.marks
        FROM marks m
        JOIN subjects sub ON m.subject_id = sub.subject_id
        WHERE m.student_id = ?
    ''', (student_id,))
    subjects = cursor.fetchall()

    if not subjects:
        print(f"No marks found for student ID {student_id}")
        return

    # Calculate total, average, grade, status
    marks_list = [row[1] for row in subjects]
    total = sum(marks_list)
    average = total / len(marks_list)

    # Grade logic
    if average >= 90:
        grade = 'A+'
    elif average >= 80:
        grade = 'A'
    elif average >= 70:
        grade = 'B'
    elif average >= 60:
        grade = 'C'
    elif average >= 50:
        grade = 'D'
    elif average >= 40:
        grade = 'E'
    else:
        grade = 'F'

    # Pass/Fail logic (all subjects must be >= 40)
    status = "Pass" if all(mark >= 40 for mark in marks_list) else "Fail"

    # Print report
    print(f"--- Report for Student ID: {student_id} ---")
    print(f"Name: {student_name}\n")
    print("Subject-wise Marks:")
    for subject, mark in subjects:
        print(f"{subject}: {mark}")
    print(f"\nTotal Marks: {total}")
    print(f"Average Marks: {average:.2f}")
    print(f"Grade: {grade}")
    print(f"Status: {status}")


    


In [246]:
#Department_summary
def department_summary():
    cursor.execute('''
        SELECT 
            d.dept_name,
            ROUND(AVG(m.marks), 2) AS average_marks
        FROM marks m
        JOIN student s ON m.student_id = s.student_id
        JOIN departments d ON s.dept_id = d.dept_id
        GROUP BY d.dept_name
    ''')

    print("Department-wise Average Marks:")
    for dept_name, avg_marks in cursor.fetchall():
        print(f"{dept_name}: {avg_marks}")



In [247]:
def show_studentsNameNumber():
    

    cursor.execute('''
        SELECT student_id, name FROM student
    ''')

    for row in cursor.fetchall():
        print(f"Student ID: {row[0]}, Name: {row[1]}")



In [248]:
print("\n------------------------------- Students-------------------------------")
show_studenst()
print("\n" + "=" * 100 + "\n")

print("\n------------------------------- Subjects -------------------------------")
show_subjects()
print("\n" + "=" * 100 + "\n")

print("\n------------------------------- Marks -------------------------------")
show_marks()
print("\n" + "=" * 100 + "\n")

print("\n------------------------------- Toppers -------------------------------")
toppers()
print("\n" + "=" * 100 + "\n")

print("\n------------------------------- Unique Subjects Are -------------------------------")
unique_subjects()

print("\n------------------------------- Department Summary -------------------------------")
department_summary()

print("\n------------------------------- Select an ID to see the result -------------------------------")
show_studentsNameNumber()
va=int(input("Select an ID to see the result"))
studentReports(va)



------------------------------- Students-------------------------------

--- Students ---
(1, 'AA', 'DL', 'Data Science')
(2, 'BB', 'ML', 'Data Science')
(3, 'CC', 'Set', 'Physics')
(4, 'DD', 'Algebra', 'Physics')
(5, 'EE', 'Calcus', 'Data Science')



------------------------------- Subjects -------------------------------

--- Subjects ---
(1, 'Data Structures', 'Data Science')
(2, 'DBMS', 'Data Science')
(3, 'Algebra', 'Physics')
(4, 'Calcus', 'Physics')



------------------------------- Marks -------------------------------

--- Marks ---
('AA', 'Data Structures', 78)
('AA', 'DBMS', 81)
('BB', 'Data Structures', 75)
('BB', 'DBMS', 80)
('CC', 'Data Structures', 50)
('CC', 'DBMS', 45)
('DD', 'Data Structures', 85)
('DD', 'DBMS', 90)
('EE', 'Data Structures', 95)
('EE', 'DBMS', 92)



------------------------------- Toppers -------------------------------

--- Topper in Each Subject ---
Subject: Data Structures 
 Topper: EE         | Marks: 95
Subject: DBMS       
 Topper: EE       

Select an ID to see the result 5


--- Report for Student ID: 5 ---
Name: EE

Subject-wise Marks:
Data Structures: 95
DBMS: 92

Total Marks: 187
Average Marks: 93.50
Grade: A+
Status: Pass
